# Python Algorithms
## Chapter 5 Traversal: The Skeleton Key of Algorithmics
+ traversal: discovering, and later visiting, all the nodes in a graph.   

finding the connected components of a graph. A graph is connected if there is a path from each node to each of the others and if the connected components are the maximal subgraphs that are (individually) connected. One way of finding a connected component would be to start 
at some place in the graph and gradually grow a larger connected subgraph until we can’t get any further. Let’s look at the following related problem. Show that you can order the nodes in a connected graph, $v_1, v_2, . .., v_n$, so that for any $i = 1.. .n$, the subgraph over $v_1, . .. , v_i$ is connected. If we can show this and we can figure out how to do 
the ordering, we can go through all the nodes in a connected component and know when they’re all used up.
### A Walk in the Park
#### No Cycles Allowed
#### How to Stop Walking in Circles
+ start walking in any direction, backtracking whenever you came to a dead end or an intersection you had already walked through
#### Go Deep!
#### Depth-First Timestamps and Topological Sorting
#### Infinite Mazes and Shortest (Unweighted) Paths
+ If we’re looking for the shortest paths (disregarding edge weights, for now) from our start node to all the others, DFS will, most likely, give us the wrong answer
+ iterative deepening depth-first search, or IDDFS, and it simply consists of running a depth-constrained DFS with an iteratively incremented depth limit
+ There is really only one situation where IDDFS would be preferable over BFS: when searching a huge tree (or some state space “shaped” like a tree). Because there are no cycles, we don’t need to remember which nodes we’ve visited, which means that IDDFS needs only store the path back to the starting node. BFS, on the other hand, must keep the entire fringe in memory (as its queue), and as long as there is some branching, this fringe will grow exponentially with the distance to the root. In other words, in these cases IDDFS can save a significant amount of memory, with little or no asymptotic slowdown.
### Strongly Connected Component
+ A connected component is a maximal subgraph where all nodes can reach each other if you ignore edge directions (or if the graph is undirected). To get strongly connected components, though, you need to follow the edge directions; so, SCCs are the maximal subgraphs where there is a directed path from any node to any other.
+ In fact, in general, if there is an edge from any strong component X to another strong component Y, the last finish time in X will be later than the latest in Y.